## This is agent based with 4 tools.

In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
host = 'localhost'                                                                                                                  
port = '3306'
username = 'root'
password = 'shivam2003'
database_schema = 'regional_sales_data'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [5]:
db = SQLDatabase.from_uri(mysql_uri)

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")

Dialect: mysql
Available tables: ['2017_budgets', 'customers', 'products', 'regions', 'sales_order', 'state_regions']


In [30]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = "API_KEY_HERE"

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")



In [31]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [32]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001503C2834F0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001503C2834F0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001503C2834F0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [33]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)

In [34]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

f:\zep2\TextTOSql\sqlagent\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


In [35]:
system_message = prompt_template.format(dialect="mysql", top_k=5)

In [36]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, toolkit.get_tools(), prompt=system_message)

In [37]:
example_query = "How many products are there in the database?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How many products are there in the database?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (733af99f-8a56-400f-b876-af7dc2d3c0b4)
 Call ID: 733af99f-8a56-400f-b876-af7dc2d3c0b4
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

2017_budgets, customers, products, regions, sales_order, state_regions
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (0500e557-4f1d-4bef-8f48-6dfd5e7b2c1f)
 Call ID: 0500e557-4f1d-4bef-8f48-6dfd5e7b2c1f
  Args:
    query: SELECT count(*) FROM products
================================= Tool Message =================================
Name: sql_db_query

[(30,)]
================================== Ai Message ==================================

There are 30 products in the database.
